In [1]:
import h5py
import ast
from astropy.table import QTable, Table, Column, vstack
import astropy.units as u
import astropy
%matplotlib inline
%matplotlib notebook
import pandas as pd
import numpy as np
import simsurvey
import simsurvey_tools as sst
import matplotlib.pylab as plt
import pandas as pd
import os
from ZTF_hdf5 import Write_LightCurve, Read_LightCurve, Plot_LightCurve

In [2]:
home_dir = os.environ.get('HOME')
sfd98_dir = os.path.join(home_dir, 'data/sfd98')
sfd98_dir = '/Users/manon/ZTF/data/sfd98'

fields = sst.load_ztf_fields()
#sst.load_ztf_filters()
ccds = sst.load_ztf_ccds(filename='data/ZTF_corners_rcid.txt', num_segs=64) #it's rcid
#ccds = sst.load_ztf_ccds() #its' ccd

obs = pd.read_csv('data/2018_all_logs_from_dr1_rcid_zp_from_masci.csv')
obs[obs['rcid']==26]
obs['field'] =obs['field'].astype('int64')
obs['time'] = obs['time'] - 2400000.5

plan = simsurvey.SurveyPlan(time=obs['time'], band=obs['band'], zp=obs['zp'], obs_field=obs['field'], obs_ccd=obs['rcid'],
                            skynoise=obs['skynoise'], fields={k: v for k, v in fields.items() if k in 
                            ['ra', 'dec', 'field_id', 'width', 'height']}, ccds=ccds)

z_range = (0.01, 0.1)
dec_range = (-30, 90)
mjd_range = (plan.cadence['time'].min()- 30, plan.cadence['time'].max() + 30) #
# we  put a ra range as well
ntransient = 11 # we can change the number of transientor the rate (by def  ntransient = len(redshifts))
seed = 70 #we can fix teh seed

tr = simsurvey.get_transient_generator(zrange=z_range, transient='Ia', template='salt2', dec_range=dec_range,
                                       mjd_range=mjd_range, sfd98_dir=sfd98_dir, ntransient=ntransient, seed=seed)

n_det = 10
survey = simsurvey.SimulSurvey(generator=tr, plan=plan, n_det=n_det)

lc_sn_default = survey.get_lightcurves(progress_bar=True, notebook=True) #simullc_func
print(len(lc_sn_default.lcs))

Determining field IDs for all objects


FloatProgress(value=0.0)


Generating lightcurves


FloatProgress(value=0.0)


5


In [212]:
clas = Write_LightCurve()
data = clas.write_data('Data', lc_sn_default)
meta = clas.write_meta()
meta

lc is not an astropy.table.table.Table type <class 'simsurvey.simulsurvey.LightcurveCollection'>
OrderedDict([('directory', '/Users/manon/dataLC/Meta.hdf5'), ('file_name', 'Meta.dhf5')])


In [213]:
clas2 = Read_LightCurve(file_name='Data.hdf5')
read_data = clas2.Read_file(path='SN_1')
read_data

time,band,flux,fluxerr,zp,zpsys,field,ccd,comment
float64,bytes4,float64,float64,float64,bytes2,int64,int64,bytes1
58198.41731479997,ztfr,-21.484466827564876,20.46585984561505,26.325,ab,430,7,
58198.459409699775,ztfr,-29.642261075747708,19.90810834703056,26.325,ab,430,7,
58203.343877300154,ztfg,-137.56552247158902,144.22149583657995,26.275,ab,430,7,
58203.35707180016,ztfg,-37.73515961802923,97.95576387368912,26.275,ab,430,7,
58206.44896990014,ztfg,9.880168958831122,40.46038357356532,26.275,ab,430,7,
...,...,...,...,...,...,...,...,...
58275.2719676001,ztfg,37.881321037010345,14.0535423202123,26.275,ab,430,7,
58275.35569440015,ztfr,136.2265633610083,24.364214275283626,26.325,ab,430,7,
58278.2704977002,ztfr,135.6236645855769,32.4596874548094,26.325,ab,430,7,


In [336]:
import csv
from astropy.table import Table
home_dir = os.environ.get('HOME')
folder_dir = os.path.join(home_dir, 'dataLC/Selection_tab.csv')
folder_dir2 = os.path.join(home_dir, 'dataLC/str_tab.csv')

table = Table.read(folder_dir) #Lecture du fichier csv
table_str = Table.read(folder_dir2) #Lecture du fichier csv

In [337]:
table_str

col,lim_col,op1,seuil,name_selec
str4,str4,str11,int64,str7
band,ztfr,operator.eq,0,nb_ztfr
band,ztfg,operator.eq,0,nb_ztfg


In [338]:
import operator
def selec_phase(Tab, csvFile, Verbose=False):
    
    if csvFile['col'] not in Tab.colnames:
        print('This column doesnt exist') 
    
    else:
        col = csvFile['col']
        op = eval(csvFile['op1'])
        lim = csvFile['lim_col']
    
        mask = op(Tab[col], lim)
        Tab = Tab[mask]
    
    if Verbose:
        print('len Tab :', len(Tab))
    
    return len(Tab)

In [339]:
def selec(Tab, csvFile, limSNR, Verbose=False,  Plot=False, selec=False):
    
    SNR = Tab['flux'] / Tab['fluxerr']
    phase = (Tab['time'] - Tab.meta['t0']) / (1-Tab.meta['z'])
    Tab.add_columns([SNR, phase], names=['SNR', 'phase'])
    mask_SNR = Tab['SNR'] >= limSNR
    Tab = Tab[mask_SNR]
    res = 1
    
    
    if Verbose:
        print('min :', np.min(Tab['phase']), 'max :', np.max(Tab['phase']))
    if Plot:
        x = Tab['phase']
        y = Tab['flux']

        fig, axs = plt.subplots(1, 1, figsize=(10,8))
        axs.plot(x,y,marker = '*')
    
    if not selec :
        selec = []    
        for i, row in enumerate(csvFile):
            len_tab = selec_phase(Tab, row)
            if res:
                res &= operator.ge(len_tab, row['seuil'])
                selec.append(row['name_selec'])
            if not res:
                break
        return selec, res
    else:
        return Tab

In [359]:
class_meta = Read_LightCurve(file_name='Meta.hdf5')
read_meta = class_meta.Read_file(path='meta')

csvTab = table
rtot = []
rb = csvTab['name_selec']   
for i, row in enumerate(read_meta):
    Tab = selec(lc_sn_default[i], csvTab, 5, selec=True)
    r = []
    for i, rowb in enumerate(csvTab):
        nb = selec_phase(Tab, rowb)
        r.append(nb)
    rtot.append(r)

rtot = np.array(rtot)
print(rtot, len(rtot))

for i in range(0,len(rtot[0])):
    line = [row[i] for row in rtot]
    c = Column(line)
    read_meta.add_column(c, name=rb[i])
read_meta

[[ 14  29   2   1]
 [  3  11   1   4]
 [  0 144   0  64]
 [  6  11   0   6]
 [ 14  48   2   9]] 5


z,t0,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,idx_orig,path,nb_phase<0,nb_phase>0,nb_phase<-10,nb_phase>20
float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,bytes4,int64,int64,int64,int64
0.09545085017442173,58371.817850549065,0.0002771357912699138,-0.62548765619564,0.08991019018243052,0.144720279260781,352.06966274388003,44.524529678477535,0.14370772544891622,1,SN_0,14,29,2,1
0.052395678566137256,58231.201247162135,0.0011922258089010543,-2.6676509956069445,-0.03448578174354965,0.2748790046966661,242.8963714073767,-5.72510622133645,0.2746621642499891,5,SN_1,3,11,1,4
0.07527202479792483,58421.5335325382,0.0005445907219979513,-0.2915194225651692,0.09710010223419693,0.028955382485282188,122.88362397997601,72.3087518839699,0.021226937444662287,7,SN_2,0,144,0,64
0.07864194667329098,58210.721340815704,0.0008432786739296758,-0.1441809256096224,-0.07964318534293403,0.028054051979465203,175.80441899319186,2.7920990514452346,0.017430438951182806,8,SN_3,6,11,0,6
0.07636610797218073,58273.41723662111,0.0004897470345753179,-0.9579709291843637,0.07850842542752962,0.027115198698569112,190.470983126126,13.051813235586533,0.023295795365418693,9,SN_4,14,48,2,9


In [360]:
R, sel_list = [], []

for i, path in enumerate(read_meta['path']):
    class_data = Read_LightCurve(file_name='Data.hdf5')
    lc = class_data.Read_file(path=path)
    sel = selec(lc, csvTab, limSNR=5, Verbose=False, Plot=False)
    R.append(sel[1])
    sel_list.append(list(sel[0]))
    print(path, sel)
    
pass_selec = Column(R)
read_meta.add_column(pass_selec, name = 'pass selec')
read_meta

SN_0 (['nb_phase<0', 'nb_phase>0', 'nb_phase<-10', 'nb_phase>20'], 1)
SN_1 (['nb_phase<0'], 0)
SN_2 (['nb_phase<0'], 0)
SN_3 (['nb_phase<0', 'nb_phase>0', 'nb_phase<-10'], 0)
SN_4 (['nb_phase<0', 'nb_phase>0', 'nb_phase<-10', 'nb_phase>20'], 1)


z,t0,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,idx_orig,path,nb_phase<0,nb_phase>0,nb_phase<-10,nb_phase>20,pass selec
float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,bytes4,int64,int64,int64,int64,int64
0.09545085017442173,58371.817850549065,0.0002771357912699138,-0.62548765619564,0.08991019018243052,0.144720279260781,352.06966274388003,44.524529678477535,0.14370772544891622,1,SN_0,14,29,2,1,1
0.052395678566137256,58231.201247162135,0.0011922258089010543,-2.6676509956069445,-0.03448578174354965,0.2748790046966661,242.8963714073767,-5.72510622133645,0.2746621642499891,5,SN_1,3,11,1,4,0
0.07527202479792483,58421.5335325382,0.0005445907219979513,-0.2915194225651692,0.09710010223419693,0.028955382485282188,122.88362397997601,72.3087518839699,0.021226937444662287,7,SN_2,0,144,0,64,0
0.07864194667329098,58210.721340815704,0.0008432786739296758,-0.1441809256096224,-0.07964318534293403,0.028054051979465203,175.80441899319186,2.7920990514452346,0.017430438951182806,8,SN_3,6,11,0,6,0
0.07636610797218073,58273.41723662111,0.0004897470345753179,-0.9579709291843637,0.07850842542752962,0.027115198698569112,190.470983126126,13.051813235586533,0.023295795365418693,9,SN_4,14,48,2,9,1
